In [1]:
import os
import sys
import json
import re
import logging

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import custom_cde as cde
from custom_cde import Document
from custom_cde.doc import Paragraph, Heading, Table
from custom_cde.reader import AcsHtmlReader
# from custom_cde.model import Compound
# from custom_cde.model import BaseModel, StringType, ListType, ModelType
# from custom_cde.parse import R, I, W, Optional, merge
# from custom_cde.parse.base import BaseParser
# from custom_cde.utils import first

%load_ext autoreload

In [4]:
#run this cell to activate debugging comments

logging.basicConfig(stream=sys.stdout, level = logging.DEBUG)

logger = logging.getLogger('LOGGER_NAME')

#test logger output
logger.debug('testing')

DEBUG:LOGGER_NAME:testing


In [5]:
#practice sentences for text parsers

d = Document(
#     Heading('Molecular energy levels of poly(3-hexylthiophene) (P3HT)'),
#     Paragraph('P3HT has a HOMO level of -4.9 eV, while the LUMO level is -3.0 eV. This corresponds to a bandgap of 1.9 eV.'),
#     Paragraph('For P3HT, EHOMO is -4.9 to -5.2 eV'),
#     Paragraph('For P3HT, ELUMO is -3.0 to -2.9 eV'),
#     Paragraph('for P3ht, the EBandGap is 1.9 to 2.3 eV'),
    
#     #what about the following sentence doesn't work, compared to the next one?
#     Paragraph(u'The gold electrode has a Fermi energy level of 5.1 eV'),
#     Paragraph(u'For gold, the Fermi energy level = 5.1 eV'),
    
#     Heading('Synthesis of 2,4,6-trinitrotoluene (3a)'),
#     Paragraph('The procedure was followed to yield a pale yellow solid (b.p. 240 °C)'),
    
    Heading('New property phrases'),
#     Paragraph('the crystallization temperature of gold was found to be ~1600 °C.'),
    Paragraph('for gold, the Tm is ~1600 °C.'),
#     Paragraph('The melting point of P3HT is 275 ºC.'),
#     Paragraph('The Tm of P3HT is 275 ºC.'),
#     Paragraph('the boiling point of niacin is 100 ºC.'),
#     Paragraph('For niacinamide, the corrosion inhibition efficiency = 90%.'),
#     Paragraph('for P3HT, the enthalpy of sublimation is 100 kJ/g'),
#     Paragraph('for P3HT, the enthalpy of vaporization is 73 J/g'),
    Paragraph('the enthalpy of crystallization of P3HT is 27 J/g'),
    Paragraph('for P3HT, the M_n is 30 kDa'),
    Paragraph('for P3HT, the Đ is 1.2.'),
#     Paragraph('For P3HT, the modulus is between 1 and 5 GPa'),
    Paragraph('for RR-P3HT, the degree of crystallinity is 45%'),
#     Paragraph('for RR-P3HT, the photoluminescence quantum yield is 45'),
)
d

<Document: 6 elements>

In [6]:
%autoreload

d.records.serialize()

[{'names': ['gold']},
 {'melting_points': [{'value': '~1600', 'units': '°C'}]},
 {'names': ['RR-P3HT'], 'crystallinity': [{'value': '45', 'units': '%'}]},
 {'names': ['P3HT'],
  'M_n': [{'value': '30', 'units': 'kDa'}],
  'dispersity': [{'value': '1.2'}]}]

In [2]:
with open('/Users/wesleytatum/Desktop/10-1021-ma201648t.htm', 'rb') as f:
    acs_doc = Document.from_file(f, readers = [AcsHtmlReader()])
    f.close()

In [4]:
acs_doc

polymer,band gap (eV),HOMO (eV),Voc (V),Jsc(mA/cm2),FF,PCE (%),ref
PFDTBT,1.9,–5.7,1.04,4.66,0.46,2.2,92
PCPDTBT,1.4,–5.3,0.62,16.2,0.55,5.5,93
PBnDT-DTffBT,1.7,–5.5,0.91,12.91,0.61,7.2,61
PDTPn-BT,2.1,–5.6,0.31,0.75,0.37,0.09,94
polymer,band gap (eV),HOMO (eV),Voc (V),Jsc(mA/cm2),FF,PCE (%),ref
Si-PCPDTBT,1.4,–5.3,0.57,17.3,0.61,5.9,5
C-PCPDTBT,1.3,–5.3,0.37,15.5,0.48,2.7,5
PSiF-DBT,1.82,–5.4,0.90,9.5,0.51,5.4,109
PDTP-DTDPP,1.13,–4.90,0.38,14.87,0.48,2.7,110
PCDTBT,1.9,–5.5,0.88,10.6,0.66,6.1,4


In [6]:
tables = {}

for el in acs_doc.elements:
    if el.id:
#         print(el)
        if 'tbl' in el.id:
            tables[el.id] = el.to_json
        
tables

{'tbl1': <bound method BaseElement.to_json of Table(id='tbl1', references=[], caption='\n Table 1. Summary of Band Gaps, HOMO Levels, and Photovoltaic Properties of Conjugated Polymers in Figure 6b \n')>,
 'tbl2': <bound method BaseElement.to_json of Table(id='tbl2', references=[], caption='\n Table 2. Summary of Band Gaps, HOMO Levels, and Photovoltaic Properties of Conjugated Polymers in Figure 7b \n')>,
 'tbl3': <bound method BaseElement.to_json of Table(id='tbl3', references=[], caption='\n Table 3. Summary of Band Gaps, HOMO Levels, and Photovoltaic Properties of Conjugated Polymers in Figure 9b \n')>,
 'tbl4': <bound method BaseElement.to_json of Table(id='tbl4', references=[], caption='\n Table 4. Summary of Band Gaps, HOMO Levels, and Photovoltaic Properties of Conjugated Polymers in Figure 10b \n')>,
 'tbl5': <bound method BaseElement.to_json of Table(id='tbl5', references=[], caption='\n Table 5. Summary of Band Gaps, HOMO/LUMO Levels, and Photovoltaic Properties of Conjugate

In [19]:
print(tables['tbl1'])

<bound method Table.serialize of Table(id='tbl1', references=[], caption='\n Table 1. Summary of Band Gaps, HOMO Levels, and Photovoltaic Properties of Conjugated Polymers in Figure 6b \n')>


In [3]:
%time detected = acs_doc.records.serialize()
# detected

CPU times: user 8min 5s, sys: 6.81 s, total: 8min 12s
Wall time: 9min 34s


In [9]:
recs_with_props = []
bgs = []

for rec in detected:
    if len(rec.values()) > 1:
#         print(rec)
#         print('\n')
        recs_with_props.append(rec)
        compound = ''
#         for name in rec['names']:
#             compound += name + ', '
#         for prop in rec.items():
#         #props are tuples of the name and a list or dict of values
#             if prop[0] == 'band_gap':
#                 prop_bg = {compound : prop[1]}
#                 bgs.append(prop_bg)
        
recs_with_props
# bgs

[{'names': ['PBnDT-DTffBT'],
  'HOMO_level': [{'value': '–5.5', 'units': 'eV'}],
  'band_gap': [{'value': '1.7', 'units': 'eV'}],
  'pce': [{'value': '7.2', 'units': '%'}],
  'ff': [{'value': '0.61'}],
  'voc': [{'value': '0.91', 'units': 'V'}],
  'jsc': [{'value': '12.91', 'units': 'mA/cm2'}]},
 {'names': ['PDTPn-BT'],
  'HOMO_level': [{'value': '–5.6', 'units': 'eV'}],
  'band_gap': [{'value': '2.1', 'units': 'eV'}],
  'pce': [{'value': '0.09', 'units': '%'}],
  'ff': [{'value': '0.37'}],
  'voc': [{'value': '0.31', 'units': 'V'}],
  'jsc': [{'value': '0.75', 'units': 'mA/cm2'}]},
 {'names': ['Si-PCPDTBT'],
  'HOMO_level': [{'value': '–5.3', 'units': 'eV'}],
  'band_gap': [{'value': '1.4', 'units': 'eV'}],
  'pce': [{'value': '5.9', 'units': '%'}],
  'ff': [{'value': '0.61'}],
  'voc': [{'value': '0.57', 'units': 'V'}],
  'jsc': [{'value': '17.3', 'units': 'mA/cm2'}]},
 {'names': ['C-PCPDTBT'],
  'HOMO_level': [{'value': '–5.3', 'units': 'eV'}],
  'band_gap': [{'value': '1.3', 'units

In [5]:
#practice table for new header and cell parsers

%autoreload

test_table_path = '/Users/wesleytatum/Desktop/post_doc/BETO/chemdataextractor-fork/ipynb/table1.htm'

with open(test_table_path, 'rb') as f:
    table_doc = Document.from_file(f)
    f.close()

table_doc

polymer,band gap (eV),HOMO (eV),Voc (V),Jsc(mA/cm2),FF,PCE (%),ref
PFDTBT,1.9,–5.7,1.04,4.66,0.46,2.2,92
PCPDTBT,1.4,–5.3,0.62,16.2,0.55,5.5,93
PBnDT-DTffBT,1.7,–5.5,0.91,12.91,0.61,7.2,61
PDTPn-BT,2.1,–5.6,0.31,0.75,0.37,0.09,94


In [6]:
%autoreload
%time table_doc.records.serialize()

CPU times: user 256 ms, sys: 1.22 ms, total: 257 ms
Wall time: 260 ms


[{'names': ['PFDTBT'],
  'HOMO_level': [{'value': '–5.7', 'units': 'eV'}],
  'band_gap': [{'value': '1.9', 'units': 'eV'}],
  'pce': [{'value': '2.2', 'units': '%'}],
  'ff': [{'value': '0.46'}],
  'voc': [{'value': '1.04', 'units': 'V'}],
  'jsc': [{'value': '4.66', 'units': 'mA/cm2'}]},
 {'names': ['PCPDTBT'],
  'HOMO_level': [{'value': '–5.3', 'units': 'eV'}],
  'band_gap': [{'value': '1.4', 'units': 'eV'}],
  'pce': [{'value': '5.5', 'units': '%'}],
  'ff': [{'value': '0.55'}],
  'voc': [{'value': '0.62', 'units': 'V'}],
  'jsc': [{'value': '16.2', 'units': 'mA/cm2'}]},
 {'names': ['PBnDT-DTffBT'],
  'HOMO_level': [{'value': '–5.5', 'units': 'eV'}],
  'band_gap': [{'value': '1.7', 'units': 'eV'}],
  'pce': [{'value': '7.2', 'units': '%'}],
  'ff': [{'value': '0.61'}],
  'voc': [{'value': '0.91', 'units': 'V'}],
  'jsc': [{'value': '12.91', 'units': 'mA/cm2'}]},
 {'names': ['PDTPn-BT'],
  'HOMO_level': [{'value': '–5.6', 'units': 'eV'}],
  'band_gap': [{'value': '2.1', 'units': 'eV'